In [1]:
#%% [markdown]

#%%
import os
print(os.getcwd())
if os.getcwd().endswith("modelling"):
    os.chdir("..")

#%%
is_cscs_run = False

if is_cscs_run:
    os.chdir("ccv1/tierli_ahluege/")
    print(os.getcwd())

#%%
from src.modelling import CCV1_Trainer
from src.augmentation import CCV1Transformer,None_Transform
from src.data_modules import DataModule
from torch import nn
import torch
from torchvision import transforms,models

#%%
# wandb Notebook Setup
%env "WANDB_NOTEBOOK_NAME" "ccv1_convnext"
%env WANDB_SILENT=True

def model_freeze(model,n_block):
    backbone = list(model.features.children())
    print(f"The Backbone has {len(backbone)} blocks")
    ct = 0
    for child in backbone:
        ct += 1
        if ct < n_block:
            for param in child.parameters():
                param.requires_grad = False
        else:
            print("No freeze")
            print(child)
# %%
def convnext_tiny_():
    model = models.convnext_tiny(weights=True,)
    model_freeze(model,7)
    model.classifier[2] = nn.Linear(in_features=768, out_features=8, bias=True)
    return model



/users/tmandelz/ccv1/tierli_ahluege/modelling


/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: "WANDB_NOTEBOOK_NAME"="ccv1_convnext"
env: WANDB_SILENT=True


In [11]:
#%%
pretrained_model = "convnext"
convnext_transformer = CCV1Transformer(
    transforms.Compose([None_Transform()]), "model_specific", pretrained_model
).getCompose(True)
convnext_transformer

#%%
pretrained_model = "convnext"
convnext_transformer_augmented = CCV1Transformer(
    transforms.Compose([transforms.AugMix()]), "model_specific", pretrained_model
).getCompose(True)
convnext_transformer_augmented


#%%
pretrained_model = "convnext"
convnext_transformer_augmented_less = CCV1Transformer(
    transforms.Compose([transforms.AugMix(2,2)]), "model_specific", pretrained_model
).getCompose(True)
convnext_transformer_augmented_less

#%%
pretrained_model = "convnext"
convnext_transformer_augmented_more = CCV1Transformer(
    transforms.Compose([transforms.AugMix(4,4)]), "model_specific", pretrained_model
).getCompose(True)
convnext_transformer_augmented_more

#%%
pretrained_model = "convnext"
convnext_transformer_augmented_lesser = CCV1Transformer(
    transforms.Compose([transforms.AugMix(1,1)]), "model_specific", pretrained_model
).getCompose(True)
convnext_transformer_augmented_lesser

Compose(
    None_Transform()
    Compose(
    AugMix(severity=1, mixture_width=1, chain_depth=-1, alpha=1.0, all_ops=True, interpolation=InterpolationMode.BILINEAR, fill=None)
)
    ToTensor()
    Compose(
    None_Transform()
)
    Compose(
    Resize(size=(236, 236), interpolation=bilinear, max_size=None, antialias=True)
    CenterCrop(size=(224, 224))
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
)
)

In [3]:
model_name = "ensemble_range_freeze_higher"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=1.01,threshhold_megadetector=0.8,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=3, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4,decrease_confidence_validation=0.9)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:39<00:00, 159.66s/fold]


In [4]:
convnext.submission(model_name,ensemble=True)

Saved model: ensemble_range_freeze_higher to ./model_submit/ensemble_range_freeze_higher.pth
Saved submission: ensemble_range_freeze_higher to ./data_submit/ensemble_range_freeze_higher.csv


In [5]:
model_name = "ensemble_range_freeze_high"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.8,threshhold_megadetector=0.60,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [01:48<00:00, 108.98s/fold]


In [6]:
convnext.submission(model_name,ensemble=True)

Saved model: ensemble_range_freeze_high to ./model_submit/ensemble_range_freeze_high.pth
Saved submission: ensemble_range_freeze_high to ./data_submit/ensemble_range_freeze_high.csv


In [11]:
model_name = "ensemble_range_freeze_range_mid"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.60,threshhold_megadetector=0.4,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=8, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:03<00:00, 123.01s/fold]


In [12]:
convnext.submission(model_name,ensemble=True)

Saved model: ensemble_range_freeze_range_mid to ./model_submit/ensemble_range_freeze_range_mid.pth
Saved submission: ensemble_range_freeze_range_mid to ./data_submit/ensemble_range_freeze_range_mid.csv


In [9]:
model_name = "ensemble_range_freeze_low"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer,delete_recognized_mega=True,threshhold_megadetector=0.4), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [04:39<00:00, 279.01s/fold]


In [10]:
convnext.submission(model_name,ensemble=True)

Saved model: ensemble_range_freeze_low to ./model_submit/ensemble_range_freeze_low.pth
Saved submission: ensemble_range_freeze_low to ./data_submit/ensemble_range_freeze_low.csv


In [14]:
model_name = "ensemble_range_freeze_higher_augmix"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=1.01,threshhold_megadetector=0.8,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=3, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4,decrease_confidence_validation=0.9)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:43<00:00, 163.33s/fold]


In [ ]:
convnext.submission(model_name,ensemble=True)

In [18]:
model_name = "ensemble_range_freeze_higher_augmix_less"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_more,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=1.01,threshhold_megadetector=0.8,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=3, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4,decrease_confidence_validation=0.9)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:36<00:00, 156.80s/fold]


In [20]:
model_name = "ensemble_range_freeze_high_augmix"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.8,threshhold_megadetector=0.60,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [01:54<00:00, 114.58s/fold]


In [3]:
model_name = "ensemble_range_freeze_high_augmix_more"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_more,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.8,threshhold_megadetector=0.60,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:02<00:00, 122.83s/fold]


In [4]:
model_name = "ensemble_range_freeze_high_augmix_less"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_less,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.8,threshhold_megadetector=0.60,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [01:53<00:00, 113.41s/fold]


In [5]:
model_name = "ensemble_range_freeze_range_mid_augmix"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.60,threshhold_megadetector=0.4,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=8, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:09<00:00, 129.23s/fold]


In [6]:
model_name = "ensemble_range_freeze_range_mid_augmix_more"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_more,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.60,threshhold_megadetector=0.4,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=8, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:09<00:00, 129.98s/fold]


In [7]:
model_name = "ensemble_range_freeze_range_mid_augmix_less"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_less,include_megadetector_train=True,delete_unrecognized_mega=True,max_threshhold_megadetector=0.60,threshhold_megadetector=0.4,include_megadetector_test=True), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=8, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:142: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  val_labels = val_labels[(val_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/ccv1/tierli_ahluege/src/data_modules.py:149: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  train_labels = train_labels[(train_features["conf"]>self.max_threshhold_megadetector)==False]
/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [02:03<00:00, 123.42s/fold]


In [8]:
model_name = "ensemble_range_freeze_low_augmix"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented,delete_recognized_mega=True,threshhold_megadetector=0.4), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [06:55<00:00, 415.84s/fold]


In [9]:
model_name = "ensemble_range_freeze_low_augmix_more"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_more,delete_recognized_mega=True,threshhold_megadetector=0.4), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [07:26<00:00, 446.13s/fold]


In [10]:
model_name = "ensemble_range_freeze_low_augmix_less"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_less,delete_recognized_mega=True,threshhold_megadetector=0.4), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=4, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [05:58<00:00, 358.13s/fold]


In [12]:
model_name = "ensemble_range_freeze_low_augmix_lesser"
convnext = CCV1_Trainer(DataModule(basic_transform=convnext_transformer_augmented_lesser,delete_recognized_mega=True,threshhold_megadetector=0.4), convnext_tiny_,)
convnext.train_model(model_name, pretrained_model, num_epochs=1, cross_validation=False,test_model=False,batchsize_train_data=64,lr = 1e-4)


Fold-Iteration:   0%|          | 0/1 [00:00<?, ?fold/s]/users/tmandelz/.local/share/virtualenvs/tierli_ahluege-KocqYYbw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ConvNeXt_Tiny_Weights.IMAGENET1K_V1`. You can also use `weights=ConvNeXt_Tiny_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


The Backbone has 8 blocks
No freeze
Sequential(
  (0): LayerNorm2d((384,), eps=1e-06, elementwise_affine=True)
  (1): Conv2d(384, 768, kernel_size=(2, 2), stride=(2, 2))
)
No freeze
Sequential(
  (0): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')
      (5): Linear(in_features=3072, out_features=768, bias=True)
      (6): Permute()
    )
    (stochastic_depth): StochasticDepth(p=0.08823529411764706, mode=row)
  )
  (1): CNBlock(
    (block): Sequential(
      (0): Conv2d(768, 768, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=768)
      (1): Permute()
      (2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (3): Linear(in_features=768, out_features=3072, bias=True)
      (4): GELU(approximate='none')


Fold-Iteration: 100%|██████████| 1/1 [01:20<00:00, 80.42s/fold]
